## Träna CBoW-modellen

Den här anteckningsboken är en del av [AI for Beginners Curriculum](http://aka.ms/ai-beginners)

I det här exemplet kommer vi att titta på att träna en CBoW-språkmodell för att skapa vår egen Word2Vec-inbäddningsrymd. Vi kommer att använda AG News-datasetet som textkälla.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Först ska vi ladda vår dataset och definiera tokenizer och ordförråd. Vi kommer att sätta `vocab_size` till 5000 för att begränsa beräkningarna något.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW-modell

CBoW lär sig att förutsäga ett ord baserat på de $2N$ närliggande orden. Till exempel, när $N=1$, får vi följande par från meningen *I like to train networks*: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). Här är det första ordet det närliggande ordet som används som input, och det andra ordet är det vi förutspår.

För att bygga ett nätverk som förutspår nästa ord behöver vi ge det närliggande ordet som input och få ordnumret som output. Arkitekturen för CBoW-nätverket är följande:

* Inputordet skickas genom inbäddningslagret. Detta inbäddningslager kommer att vara vår Word2Vec-inbäddning, så vi definierar det separat som variabeln `embedder`. Vi kommer att använda en inbäddningsstorlek på 30 i detta exempel, även om du kanske vill experimentera med högre dimensioner (riktig Word2Vec har 300).
* Inbäddningsvektorn skickas sedan till ett linjärt lager som förutspår outputordet. Därför har det `vocab_size` neuroner.

För output, om vi använder `CrossEntropyLoss` som förlustfunktion, behöver vi också bara ge ordnummer som förväntade resultat, utan one-hot-kodning.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## Förbereda träningsdata

Nu ska vi programmera huvudfunktionen som kommer att beräkna CBoW-ordpar från text. Den här funktionen låter oss specificera fönsterstorlek och kommer att returnera ett set av par - inmatnings- och utmatningsord. Observera att denna funktion kan användas på ord, såväl som på vektorer/tensorer - vilket gör det möjligt för oss att koda texten innan vi skickar den till funktionen `to_cbow`.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


Låt oss förbereda träningsdatasetet. Vi kommer att gå igenom alla nyheter, kalla `to_cbow` för att få listan över ordpar, och lägga till dessa par till `X` och `Y`. För att spara tid kommer vi endast att överväga de första 10k nyhetsartiklarna - du kan enkelt ta bort begränsningen om du har mer tid att vänta och vill få bättre inbäddningar :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

Vi kommer också att konvertera den datan till en dataset och skapa en dataloader:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

Vi kommer också att konvertera den datan till en dataset och skapa en dataloader:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

Nu ska vi göra den faktiska träningen. Vi kommer att använda `SGD`-optimeraren med ganska hög inlärningshastighet. Du kan också prova att experimentera med andra optimerare, såsom `Adam`. Vi kommer att träna i 10 epoker till att börja med - och du kan köra om denna cell om du vill ha ännu lägre förlust.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Prova Word2Vec

För att använda Word2Vec, låt oss extrahera vektorer som motsvarar alla ord i vårt ordförråd:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

Låt oss se, till exempel, hur ordet **Paris** kodas till en vektor:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Det är intressant att använda Word2Vec för att leta efter synonymer. Följande funktion kommer att returnera `n` närmaste ord till ett givet input. För att hitta dem beräknar vi normen av $|w_i - v|$, där $v$ är vektorn som motsvarar vårt inputord, och $w_i$ är kodningen av det $i$:te ordet i vokabulären. Vi sorterar sedan arrayen och returnerar motsvarande index med hjälp av `argsort`, och tar de första `n` elementen i listan, som kodar positionerna för de närmaste orden i vokabulären.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## Slutsats

Genom att använda smarta tekniker som CBoW kan vi träna en Word2Vec-modell. Du kan också prova att träna en skip-gram-modell som är utformad för att förutsäga närliggande ord baserat på det centrala ordet och se hur bra den presterar.



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet, bör du vara medveten om att automatiserade översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess ursprungliga språk bör betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
